In [ ]:
from lucifex.solver import OptionsPETSc
from lucifex.fdm import AB, CN
from lucifex.sim import run, Simulation
from lucifex.viz import plot_colormap, plot_line
from crocodil.dns import dns_system_a, SYSTEM_A_REFERENCE

STORE = 1
NX = 100
NY = 100
RA = 1000.0

def create_simulation(
    Da: float,
):
    physical = dict(SYSTEM_A_REFERENCE)
    physical.update(Da=Da)
    return dns_system_a(
        store_delta=STORE, 
    )(
        Nx=NX,
        Ny=NY,
        scaling='advective',
        **physical,
        D_adv=AB(1)@CN,
        D_diff=AB(1)@CN,
        dt_max=0.1,
        cfl_courant=0.5,
        c_stabilization=None,
        c_limits=None,
        c_petsc=OptionsPETSc('gmres', 'ilu'),
        flow_petsc=(OptionsPETSc('cg', 'hypre'), None),
        diagnostic=True,
    )


Da_opts = (10.0, 100.0, 1000.0)
simulations: dict[float, Simulation] = {}

for Da in Da_opts:
    simulations[Da] = create_simulation(Da)


n_stop = 20
t_stop = 12.0
dt_init = 1e-6
n_init = 10
for Da in Da_opts:
    sim = simulations[Da]
    run(sim, n_stop, t_stop, dt_init=dt_init, n_init=n_init)